In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import parselmouth
from parselmouth.praat import call

def measure_pitch_jitter_shimmer(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID)
    

    pitch = call(sound, "To Pitch", 0.0, f0min, f0max)  
    meanF0 = call(pitch, "Get mean", 0, 0, unit)  
    stdevF0 = call(pitch, "Get standard deviation", 0, 0, unit)  
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)

    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3) * 100 
    localShimmer = call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6) * 100 

    return meanF0, stdevF0, hnr, localJitter, localShimmer


dataset_dir = r"C:\Users\DELL\Desktop\UoE\MLP\Coursework 3\Dataset 2" 

results = []

for phonation_folder in os.listdir(dataset_dir):
    phonation_path = os.path.join(dataset_dir, phonation_folder)

    if os.path.isdir(phonation_path): 
        phonation_type = phonation_folder.replace("phonation", "")

        for wave_file in glob.glob(os.path.join(phonation_path, "*.wav")):
            file_name = os.path.basename(wave_file) 
            person_id = file_name.split("_")[0]
            meanF0, stdevF0, hnr, localJitter, localShimmer = measure_pitch_jitter_shimmer(wave_file, 75, 500, "Hertz")

            results.append([person_id, phonation_type, meanF0, stdevF0, hnr, localJitter, localShimmer])

df_results = pd.DataFrame(results, columns=["ID", "Phonation", "meanF0Hz", "stdevF0Hz", "HNR", "Jitter", "Shimmer"])

df_pivot = df_results.pivot(index='ID', columns='Phonation')

df_pivot.columns = [f"{col[0]}_{col[1]}" for col in df_pivot.columns]

df_pivot.reset_index(inplace=True)

final_columns = [
    "ID",
    "meanF0Hz_A", "stdevF0Hz_A", "HNR_A", "Jitter_A", "Shimmer_A",
    "meanF0Hz_E", "stdevF0Hz_E", "HNR_E", "Jitter_E", "Shimmer_E",
    "meanF0Hz_I", "stdevF0Hz_I", "HNR_I", "Jitter_I", "Shimmer_I",
    "meanF0Hz_O", "stdevF0Hz_O", "HNR_O", "Jitter_O", "Shimmer_O",
    "meanF0Hz_U", "stdevF0Hz_U", "HNR_U", "Jitter_U", "Shimmer_U"
]

df_final = df_pivot[[col for col in final_columns if col in df_pivot.columns]]

# Save the transformed dataset to Excel
output_path = os.path.join(dataset_dir, "speech_features_pivoted.xlsx")
df_final.to_excel(output_path, index=False, engine="openpyxl")



In [2]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import ks_2samp, spearmanr, pearsonr, wasserstein_distance

def load_data(file1_path, file2_path):
    df1 = pd.read_excel(file1_path, engine="openpyxl")
    df2 = pd.read_excel(file2_path, engine="openpyxl", skiprows=[0])

    df1["ID"] = df1["ID"].astype(str).str.strip()
    df2["ID"] = df2["ID"].astype(str).str.strip()

    df1.columns = df1.columns.str.strip().str.lower()
    df2.columns = df2.columns.str.strip().str.lower()

    df_merged = df1.merge(df2, on="id", suffixes=("_new", "_old"), how="inner")

    feature_columns = [col[:-4] for col in df_merged.columns if col.endswith("_old")]

    return df_merged, feature_columns

def compute_metrics(df, features):
    results = []

    for feature in features:
        old_col = f"{feature}_old"
        new_col = f"{feature}_new"

        if old_col not in df.columns or new_col not in df.columns:
            print(f"Warning: {old_col} or {new_col} not found in DataFrame. Skipping...")
            continue

        y_true = df[old_col].dropna()
        y_pred = df[new_col].dropna()

        if y_true.nunique() == 1 or y_pred.nunique() == 1:
            ks_stat = float("nan") 
        else:
            ks_stat = ks_2samp(y_true, y_pred, method="asymp").statistic 

        mae = mean_absolute_error(y_true, y_pred)
        mse = mean_squared_error(y_true, y_pred)
        mape = (abs(y_true - y_pred) / y_true.replace(0, 1)).mean() * 100 
        pearson_corr, _ = pearsonr(y_true, y_pred)
        spearman_corr, _ = spearmanr(y_true, y_pred)
        wasserstein_dist = wasserstein_distance(y_true, y_pred)

        results.append([feature, mae, mse, mape, ks_stat, pearson_corr, spearman_corr, wasserstein_dist])

    df_results = pd.DataFrame(results, columns=[
        "Feature", "MAE", "MSE", "MAPE (%)", "KS Statistic", 
        "Pearson Correlation", "Spearman Correlation", "Wasserstein Distance"
    ])
    
    return df_results


file1_path = r"C:\Users\DELL\Desktop\UoE\MLP\Coursework 3\Dataset 2\speech_features_pivoted.xlsx"
file2_path = r"C:\Users\DELL\Desktop\UoE\MLP\Coursework 3\Dataset 2\VOC-ALS.xlsx"

df_merged, feature_columns = load_data(file1_path, file2_path)
df_errors = compute_metrics(df_merged, feature_columns)

from IPython.display import display
display(df_errors)

# Save to an Excel file for review
df_errors.to_excel("comparison_results.xlsx", index=False, engine="openpyxl")

print("Comparison results saved as 'comparison_results.xlsx'.")



,Feature,MAE,MSE,MAPE (%),KS Statistic,Pearson Correlation,Spearman Correlation,Wasserstein Distance
0,meanf0hz_a,6.908502e-10,7.526351e-18,4.746866e-10,0.019608,1.0,1.0,6.908502e-10
1,stdevf0hz_a,5.291291e-10,5.503616e-18,9.647530e-09,0.019608,1.0,1.0,5.291291e-10
2,hnr_a,6.129012e-14,6.590154e-27,3.518022e-13,0.019608,1.0,1.0,6.129012e-14
3,meanf0hz_e,1.899555e-09,3.444720e-17,1.277651e-09,0.013072,1.0,1.0,1.899555e-09
4,stdevf0hz_e,1.765389e-09,3.841374e-17,3.586700e-08,0.013072,1.0,1.0,1.765389e-09
5,hnr_e,8.870174e-14,6.845184e-26,4.390992e-13,0.013072,1.0,1.0,8.870174e-14
6,meanf0hz_i,1.666162e-09,2.206585e-17,1.071501e-09,0.013072,1.0,1.0,1.666162e-09
7,stdevf0hz_i,1.269646e-09,1.192125e-17,1.881079e-08,0.013072,1.0,1.0,1.269646e-09
8,hnr_i,6.612575e-13,1.577476e-23,2.413124e-12,0.013072,1.0,1.0,6.612575e-13
9,meanf0hz_o,5.193882e-10,5.469970e-18,3.738435e-10,0.013072,1.0,1.0,5.193882e-10


Comparison results saved as 'comparison_results.xlsx'.
